In [ ]:
#@title Importing modules
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
import pandas as pd
#from emcee.autocorr import function_1d
#import emcee
import matplotlib.pyplot as plt
import gillespy2
import os
from scipy.sparse import csr_matrix # For sparse matrices
from typing import List
from scipy.sparse.linalg import expm_multiply
from tqdm.auto import tqdm
#from joblib import Parallel, delayed
import seaborn as sns
import shutil
import matplotlib.ticker as ticker
figSize=800
#import math
import pathlib
#from collections import defaultdict

# current directory
current_dir = pathlib.Path().absolute()

# if folder_outputs doesnt exist, create it
folder_outputs = current_dir.joinpath('Figures_Exercise')

# Removing directory if exist
if os.path.exists(folder_outputs):
    shutil.rmtree(folder_outputs)
# Creating the directory
folder_outputs.mkdir(parents=True, exist_ok=True)



# Model

| Index | Reaction | Description | Reaction Rate |
|-------|----------|-------------|---------------|
| $r_1$ | $G_{off} \xrightarrow[]{k_{on}} G_{on}$ | gene activation | $k_{on} \cdot [G_{off}]$ |
| $r_2$ | $G_{on} \xrightarrow[]{k_{off}} G_{off}$ | gene deactivation | $k_{off} \cdot [G_{on}]$ |
| $r_3$ | $G_{on} \xrightarrow[]{k_{r}} G_{on} + R_{n}$ | active gene produces mRNA | $k_{r} \cdot [G_{on}]$ |
| $r_4$ | $R_{n} \xrightarrow[]{k_{t}} R_{c}$ | mRNA transport to cytoplasm | $k_{t} \cdot [R_{n}]$ |
| $r_5$ | $R_{c} \xrightarrow[]{k_{p}} R_{c} + P$ | mRNA in cytoplasm produces Protein | $k_{p} \cdot [R_{c}]$ |
| $r_6$ | $R_{n} \xrightarrow[]{\gamma_{r_n}} \phi$ | nuclear mRNA decay | $\gamma_{r_n} \cdot [R_{n}]$ |
| $r_7$ | $R_{c} \xrightarrow[]{\gamma_{r_c}} \phi$ | cytoplasm mRNA decay | $\gamma_{r_c} \cdot [R_{c}]$ |
| $r_8$ | $P \xrightarrow[]{\gamma_p} \phi$ | protein decay | $\gamma_p \cdot [P]$ |



In [ ]:
#@title Plotting configuration
plt.rcParams.update({
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 10,
})
# Assign colors to each species for plotting
colors = [ '#FBD148', '#6BCB77', '#AA66CC','#FF6B6B', '#4D96FF']
species_colors = {
    'G_off': colors[0],
    'G_on': colors[1],
    'R_n': colors[2],
    'R_c': colors[3],
    'P':  colors[4]
}



In [ ]:
#@title Plotting ODE model
def plotting_deterministic(time,concentrations_species,species_colors,drug_application_time=None,ylim_val=False,save_figure=True,plot_name='det.png'):
    plt.figure(figsize=(8, 5))
    for species, color in species_colors.items():
        plt.plot(time, concentrations_species[species], color=color, label=species,lw=4)
    if not drug_application_time is None:
        plt.axvline(x=drug_application_time, color='k', linestyle='--', label= r'$t_{drug}$',lw=1.5)
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Deterministic Dynamics')
    plt.legend(loc='upper right')
    plt.grid(True, linestyle='--', alpha=0.7)
    if ylim_val:
        plt.ylim(0,ylim_val)
    if save_figure == True: 
        plt.savefig(folder_outputs.joinpath(plot_name), dpi=300, bbox_inches='tight', transparent=True, pad_inches=0.1)  
    plt.show()

In [ ]:
#@title Plotting SSA model
def plotting_stochastic(time, trajectories_species,species_colors,drug_application_time=None,ylim_val=False,save_figure=True,plot_name='ssa.png'):
    def plot_species_trajectories(time, trajectories_species, species_name, color):
        # Extract the trajectories for the species
        trajectories = trajectories_species[species_name]
        # Calculate the mean and standard deviation across all trajectories
        mean_trajectories = np.mean(trajectories, axis=0)
        std_trajectories = np.std(trajectories, axis=0)
        # Plot mean concentration with standard deviation as shaded area
        plt.plot(time, mean_trajectories, '-', color=color, label=species_name, lw=4)
        plt.fill_between(time, mean_trajectories - std_trajectories, mean_trajectories + std_trajectories, color=color, alpha=0.1)
    plt.figure(figsize=(8, 5))
    # Plot each species
    for species, color in species_colors.items():
        plot_species_trajectories(time, trajectories_species, species, color)
    # Mark the drug application time
    if not drug_application_time is None:
        plt.axvline(x=drug_application_time, color='k', linestyle='--', label=r'$t_{drug}$', lw=1.5)
    # Set plot details
    plt.xlabel('Time')
    plt.ylabel('Number of Molecules')
    plt.title('Stochastic Dynamics')
    plt.legend(loc='upper right')
    plt.grid(True, linestyle='--', alpha=0.7)
    if ylim_val:
        plt.ylim(0,ylim_val)
    if save_figure == True: 
        plt.savefig(folder_outputs.joinpath(plot_name), dpi=300, bbox_inches='tight', transparent=True, pad_inches=0.1)  
    plt.show()

In [ ]:
def plotting_stochastic_dist(time, trajectories_species, species_name, species_color_dist, time_points, ylim_val=False,save_figure=True,plot_name='ssa_dist.png'):
    def plot_species_trajectories(ax, time, trajectories_species, species_name, color, time_point):
        # Extract the trajectories for the species at the specific time point
        trajectories = trajectories_species[species_name][:, time_point]
        # Plot distribution of species count at the specified time point
        ax.hist(trajectories, bins=20, color=color, alpha=0.7, label=f"{species_name} at time={time[time_point]}")
        ax.legend(loc='upper right')
    # Set up a 2x2 subplot grid
    fig, axs = plt.subplots(2, 2, figsize=(8, 8))
    axs = axs.flatten()  # Flatten the 2x2 array to easily loop over it

    # Plot the species distribution at each of the four user-selected time points
    for i, time_point in enumerate(time_points):
        plot_species_trajectories(axs[i], time, trajectories_species, species_name, species_color_dist, time_point)
        axs[i].set_xlabel('Number of Molecules')
        axs[i].set_ylabel('Frequency')
        if ylim_val:
            axs[i].set_ylim(0, ylim_val)
    # Set a general title
    plt.suptitle(f'Distribution of {species_name} at Different Time Points')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust subplots to fit into the figure area
    if save_figure == True: 
        plt.savefig(folder_outputs.joinpath(plot_name), dpi=300, bbox_inches='tight', transparent=True, pad_inches=0.1)  
    plt.show()

In [ ]:
def plotting_stochastic_dist_all_species(time, trajectories_species, species_colors, time_point, ylim_val=False, xlim_vals=None, save_figure=True, plot_name='ssa_dist_all_species.png'):
    n_species = len(species_colors)  # Number of species to plot
    n_cols = n_species  # Number of columns in subplot grid
    n_rows = 1

    def plot_species_trajectories(ax, time, trajectories_species, species_name, color, time_point, xlim_val=None):
        # Extract the trajectories for the species at the specific time point
        trajectories = trajectories_species[species_name][:, time_point]
        # Plot distribution of species count at the specified time point
        num_bins = 30  # Number of bins for the histogram
        if xlim_val:
            bins = np.linspace(xlim_val[0], xlim_val[1], num_bins + 1)
        else:
            bins = num_bins  # Use matplotlib's default bin calculation
        # Plot distribution of species count at the specified time point using defined bins
        ax.hist(trajectories, bins=bins, color=color, alpha=0.7, label=f"{species_name}")  # Label includes species name
        #ax.hist(trajectories, bins=30, color=color, alpha=0.7, label=f"{species_name}") # at time={time[time_point]}")
        ax.legend(loc='upper right')
        ax.set_xlabel('Number of Molecules')
        ax.set_ylabel('Frequency')
        if ylim_val:
            ax.set_ylim(0, ylim_val)
        if xlim_val:  # Set the x-axis limit if specified
            ax.set_xlim(xlim_val)

    # Set up subplot grid
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(16, 3 * n_rows), squeeze=False)
    axs = axs.flatten()  # Flatten the array to easily loop over it

    # Plot the species distribution at the selected time point
    for i, (species_name, species_color) in enumerate(species_colors.items()):
        xlim_val = xlim_vals.get(species_name, None) if xlim_vals else None  # Get the x-axis limit for the species if specified
        plot_species_trajectories(axs[i], time, trajectories_species, species_name, species_color, time_point, xlim_val)

    for ax in axs[i+1:]:  # Hide unused subplots
        ax.set_visible(False)

    # Set a general title
    plt.suptitle(f'Distribution of Species at Time={time[time_point]}',fontsize=20 )
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    if save_figure == True: 
        plt.savefig(folder_outputs.joinpath(plot_name), dpi=300, bbox_inches='tight', transparent=True, pad_inches=0.1)  
    plt.show()
        


In [ ]:
def plotting_combined(time, trajectories_species, species_colors, drug_application_time=None, ylim_val=None, time_point=10, save_figure=True, plot_name='ssa_tc_dist.png'):
    fig, axs = plt.subplots(2, 3, figsize=(18, 10), squeeze=False)
    
    def plot_time_courses(ax, time, trajectories_species, species_colors, drug_application_time):
        # Assuming trajectories_species is a list of dicts, each dict representing a single "trajectory" 
        # and keys are species names pointing to a 2D array (trajectories, time_points)
        for species_name, color in species_colors.items():
            # Aggregate data across all trajectories for the species
            all_trajectories = np.vstack([trajectory[species_name] for trajectory in trajectories_species])
            mean_trajectories = np.mean(all_trajectories, axis=0)
            std_trajectories = np.std(all_trajectories, axis=0)
            
            ax.plot(time, mean_trajectories, '-', color=color, label=species_name, lw=2)
            ax.fill_between(time, mean_trajectories - std_trajectories, mean_trajectories + std_trajectories, color=color, alpha=0.1)
            
            if drug_application_time is not None:
                ax.axvline(x=drug_application_time, color='k', linestyle='--', label='Drug application', lw=1.5)
            ax.set_xlabel('Time')
            ax.set_ylabel('Number of Molecules')
            ax.legend(loc='upper right', fontsize=14)
            ax.set_title('Time Courses for All Species')
            if ylim_val:
                ax.set_ylim(0, ylim_val)

    def plot_species_distribution(ax, time, trajectories_species, species_name, color, time_point):
        # Aggregate data across all trajectories for the species at the specified time point
        all_trajectories_at_time_point = np.hstack([trajectory[species_name][ time_point] for trajectory in trajectories_species])
        ax.hist(all_trajectories_at_time_point, bins=30, color=color, alpha=0.7, label=f"{species_name}")
        ax.legend(loc='upper right', fontsize=18)
        ax.set_xlabel('Number of Molecules')
        ax.set_ylabel('Frequency')
        ax.set_title(f'Distribution at Time={time[time_point]}')
    
    plot_time_courses(axs[0, 0], time, trajectories_species, species_colors, drug_application_time)
    
    species_list = list(species_colors.keys())
    for i, species_name in enumerate(species_list[:5]):  # Adjust based on the number of species and subplot slots
        col = (i + 1) % 3
        row = (i + 1) // 3
        plot_species_distribution(axs[row, col], time, trajectories_species, species_name, species_colors[species_name], time_point)
    
    for j in range(i + 2, 6):
        axs.flatten()[j].set_visible(False)
    

    plt.tight_layout()
    if save_figure == True: 
        plt.savefig(folder_outputs.joinpath(plot_name), dpi=300, bbox_inches='tight', transparent=True, pad_inches=0.1)  
    plt.show()


In [ ]:
class GeneExpressionModel(gillespy2.Model):
    def __init__(self, parameter_values,initial_conditions,mode):
        super(GeneExpressionModel, self).__init__('GeneExpressionModel')
        # Create and add parameters to the model
        for name, expression in parameter_values.items():
            self.add_parameter(gillespy2.Parameter(name=name, expression=expression))
        # Species
        G_off = gillespy2.Species(name='G_off', initial_value=initial_conditions.get('G_off') , mode=mode)
        G_on = gillespy2.Species(name='G_on', initial_value=initial_conditions.get('G_on') , mode=mode)
        R_n = gillespy2.Species(name='R_n', initial_value=initial_conditions.get('R_n') , mode=mode)
        R_c = gillespy2.Species(name='R_c', initial_value=initial_conditions.get('R_c') , mode=mode)
        P = gillespy2.Species(name='P', initial_value=initial_conditions.get('P') , mode=mode)
        self.add_species([G_off, G_on, R_n, R_c, P])
        # Reactions
        react1 = gillespy2.Reaction(name='gene_activation', reactants={G_off: 1}, products={G_on: 1}, rate=self.listOfParameters['k_on'])
        react2 = gillespy2.Reaction(name='gene_deactivation', reactants={G_on: 1}, products={G_off: 1}, rate=self.listOfParameters['k_off'])
        react3 = gillespy2.Reaction(name='mRNA_production', reactants={G_on: 1}, products={G_on: 1, R_n: 1}, rate=self.listOfParameters['k_r'])
        react4 = gillespy2.Reaction(name='mRNA_transport', reactants={R_n: 1}, products={R_c: 1}, rate=self.listOfParameters['k_t'])
        react5 = gillespy2.Reaction(name='protein_production', reactants={R_c: 1}, products={R_c: 1, P: 1}, rate=self.listOfParameters['k_p'])
        react6 = gillespy2.Reaction(name='nuclear_mRNA_decay', reactants={R_n: 1}, products={}, rate=self.listOfParameters['gamma_rn'])
        react7 = gillespy2.Reaction(name='cytoplasm_mRNA_decay', reactants={R_c: 1}, products={}, rate=self.listOfParameters['gamma_rc'])
        react8 = gillespy2.Reaction(name='protein_decay', reactants={P: 1}, products={}, rate=self.listOfParameters['gamma_p'])
        self.add_reaction([react1, react2, react3, react4, react5, react6, react7, react8])
        # Set the timespan for the model
        self.timespan(np.linspace(0, 100, 101))

def initialize_model(parameter_values,initial_conditions,mode='discrete', apply_drug =False,inhibited_parameters=None):
    model = GeneExpressionModel(parameter_values,initial_conditions,mode=mode)
    for species in model.listOfSpecies.values():
        species.mode = mode  # Dynamically set the species mode options are 'discrete' or 'continuous'
    if apply_drug ==True and not (inhibited_parameters is None):
      #apply_drug_effect
      for param, value in inhibited_parameters.items():
        if param in model.listOfParameters:
            model.listOfParameters[param].expression = str(value)
    return model


In [ ]:
def run_simulation_phase(model_initializer, parameter_values, initial_conditions, simulation_end, number_of_trajectories, apply_drug=False, inhibited_parameters=None, simulation_type='discrete', burn_in_time=None):
    """
    Run a simulation phase, including a specified burn-in time, but return results only after the burn-in period.
    """
    # Initialize the model
    model = model_initializer(parameter_values, initial_conditions, mode=simulation_type, apply_drug=apply_drug, inhibited_parameters=inhibited_parameters)
    # Set the timespan for the entire simulation phase, including any burn-in time.
    total_timespan = np.linspace(0, simulation_end, num=int(simulation_end) + 1)
    model.timespan(total_timespan)
    # Run the model
    list_all_results=[]
    trajectories_species = {}
    if simulation_type == 'discrete':
        results = model.run(solver=gillespy2.TauLeapingSolver, number_of_trajectories=number_of_trajectories)
        # For discrete simulations, gather results across all trajectories
        for n in range(number_of_trajectories):
            # Collect all trajectories for the current species
            species_trajectories = {species: results[n][species] for species in model.listOfSpecies.keys()}
            # If there's a burn-in time, exclude it from the results
            if burn_in_time is not None:
                # Assuming each unit in `total_timespan` corresponds to one simulation step
                burn_in_index = burn_in_time if burn_in_time < len(total_timespan) else len(total_timespan) - 1
                species_trajectories = {species: species_trajectories[species][burn_in_index:] for species in model.listOfSpecies.keys()}
            list_all_results.append(species_trajectories)
        return list_all_results
    else:
        # For continuous simulations, the handling remains as before
        result = model.run(solver=gillespy2.ODESolver)
        if burn_in_time is not None:
            trajectories_species = {species: result[0][species][burn_in_time:] for species in model.listOfSpecies.keys()}
        else:
            trajectories_species = {species: result[0][species] for species in model.listOfSpecies.keys()}
        return trajectories_species


In [ ]:
def simulate_model(parameter_values, initial_conditions, total_simulation_time, simulation_type, burn_in_time=None, drug_application_time=None, inhibited_parameters=None, number_of_trajectories=1):
    """
    Simulate a model either as deterministic or stochastic based on user input,
    including optional burn-in and drug application phases.
    """
    # Determine the end time for the initial simulation phase
    # This phase may include the burn-in period (if any) and extends up to the drug application time or the total simulation time
    if burn_in_time is None or burn_in_time < 50:
        burn_in_time = None
    if burn_in_time is not None:
        end_time_initial_phase = drug_application_time + burn_in_time if drug_application_time is not None else total_simulation_time+burn_in_time
    else:
        end_time_initial_phase = drug_application_time if drug_application_time is not None else total_simulation_time
    # The run_simulation_phase function will only return results after the burn-in period, adjusting the timespan accordingly
    trajectories_initial = run_simulation_phase( initialize_model, parameter_values=parameter_values, initial_conditions=initial_conditions, simulation_end=end_time_initial_phase, number_of_trajectories=number_of_trajectories, apply_drug=False,inhibited_parameters= None, simulation_type=simulation_type, burn_in_time=burn_in_time)  
    # If there's a drug application phase
    if drug_application_time is not None:
        drug_simulation_end = total_simulation_time - drug_application_time
        if simulation_type == 'continuous':
            updated_initial_conditions = {species: trajectories_initial[species][-1] for species in trajectories_initial}
            trajectories_drug = run_simulation_phase( initialize_model, parameter_values=parameter_values, initial_conditions=updated_initial_conditions, simulation_end=drug_simulation_end, number_of_trajectories=1, apply_drug=True, inhibited_parameters=inhibited_parameters, simulation_type=simulation_type)
            trajectories_species = {species: np.concatenate([trajectories_initial[species], trajectories_drug[species][1:]])
                                    for species in trajectories_initial}
        else:
            all_results_after_drug = []
            for i in range(number_of_trajectories):
                updated_initial_conditions = {species: trajectories_initial[i][species][-1] for species in trajectories_initial[i].keys()}
                trajectories_drug = run_simulation_phase( initialize_model, parameter_values=parameter_values, initial_conditions=updated_initial_conditions, simulation_end=drug_simulation_end, number_of_trajectories=1, apply_drug=True, inhibited_parameters=inhibited_parameters, simulation_type=simulation_type)
                all_results_after_drug.append(trajectories_drug[0])
            # append the results from the first phase to the results from the second phase
            trajectories_species = {}
            #if len(trajectories_initial) > 0 and len(all_results_after_drug) > 0:
            for species in trajectories_initial[0].keys():
                species_data_across_trajectories = []
                for i in range(number_of_trajectories):
                    initial_data = trajectories_initial[i][species]
                    after_drug_data = all_results_after_drug[i][species]
                    concatenated_data = np.concatenate([initial_data[:-1], after_drug_data])
                    species_data_across_trajectories.append(concatenated_data)
                trajectories_species[species] = np.stack(species_data_across_trajectories)
    else:
        trajectories_species = trajectories_initial
    # creating a vector for time span
    time = np.linspace(0, total_simulation_time, num=total_simulation_time + 1)
    return time, trajectories_species


## Adjusting $(k_t)$ to Incorporate Diffusion and Transport Rates from Simulated Cell Model

To adjust the parameter $(k_t)$, accounting for both the diffusion rate $(k_{diff_r})$ and the transport rate $(transport\_rate)$ from from Simulated Cell Model, we follow a multi-step process. This process aims to encapsulate the combined effects of RNA diffusion to the nuclear envelope and its subsequent transport into the cytosol.

### Step 1: Effective Diffusion Time $(T_{diff})$

First, we estimate the effective time $(T_{diff})$ it takes for an RNA molecule to diffuse to the nuclear envelope. Assuming a known average distance $(D)$ to the nuclear envelope, and given the diffusion rate $(k_{diff_r})$, the diffusion time can be approximated as:

$$T_{diff} = \frac{D^2}{2 \cdot k_{diff_r}}$$

This formula simplifies the complexity of cellular environments and assumes a direct, unobstructed path to the nuclear envelope.

### Step 2: Effective Transport Time $(T_{transport})$

Next, we define the effective time $(T_{transport})$ for RNA to be transported across the nuclear envelope, utilizing the transport rate $(transport\_rate)$:

$$T_{transport} = \frac{1}{transport\_rate}$$

### Step 3: Combining Effective Times

The total effective time $(T_{total})$ for an RNA molecule to move from the nucleus to the cytosol combines both diffusion and transport times:

$$T_{total} = T_{diff} + T_{transport}$$

### Step 4: Calculating Effective Rate $(k_t)$

Finally, we calculate an effective rate $(k_t)$ for RNA transport from the nucleus to the cytosol in Model 1, reflecting both diffusion and transport:

$$k_t = \frac{1}{T_{total}} = \frac{1}{\frac{D^2}{2 \cdot k_{diff_r}} + \frac{1}{transport\_rate}}$$

This equation provides a way to adjust $(k_t)$ in Model 1 to more accurately reflect the complex process of RNA movement within the cell, considering both its diffusion to the nuclear envelope and the subsequent transport into the cytosol.



In [ ]:
def calculate_effective_kt(D, k_diff_r, transport_rate, model_type):
    """
    Calculate the effective rate (k_t) for RNA transport from the nucleus to the cytosol,
    incorporating the effects of diffusion to the nuclear envelope and transport across it,
    adjusted for the dimensionality of the simulation (2D or 3D).

    Parameters:
    - D (float): Distance to the nuclear envelope (assumed to be in the same units as used for k_diff_r).
    - k_diff_r (float): Diffusion rate of RNA to the nuclear envelope.
    - transport_rate (float): Rate of RNA transport from the nucleus to the cytosol.
    - model_type: The dimensionality of the simulation (2D or 3D).

    Returns:
    - k_t (float): Effective rate of RNA transport from nucleus to cytosol.
    """
    # converting diameter to radius
    D = D/2
    # Adjust the diffusion calculation based on dimensionality
    if model_type == '2D':
        # In 2D, diffusion might be more efficient due to less volume to cover
        T_diff = D**2 / (4 * k_diff_r)  # Adjusted for 2D
    elif model_type == '3D':
        # In 3D, diffusion covers more volume
        T_diff = D**2 / (6 * k_diff_r)  # Adjusted for 3D
    else:
        raise ValueError("Dimension must be 2 or 3.")

    # Calculate T_transport as the inverse of transport_rate
    T_transport = 1 / transport_rate
    
    # Calculate the total effective time (T_total)
    T_total = T_diff + T_transport
    
    # Calculate the effective rate (k_t) as the inverse of T_total
    k_t = 1 / T_total
    
    return k_t


# Solving unperturbed moodel
___

In [ ]:
# Model Parameters
k_on = 0.5
k_off = 0.1
k_r = 3
k_p = 0.9
gamma_r = 0.05 # assuming gamma_rn = gamma_rc
gamma_p = 0.65
transport_rate = 1
diffusion_rate = 10 # assuming diffusion_rate_r = diffusion_rate_p
total_simulation_time = 201
nucleus_diameter = 40
number_of_trajectories = 50
cytosol_diameter = 70 
model_type = '2D'


# Calculating effective transport rate
effective_k_t = calculate_effective_kt(D=nucleus_diameter, k_diff_r=diffusion_rate, transport_rate=transport_rate, model_type=model_type) 
print('Effective transport rate: ', effective_k_t)
kt = effective_k_t
drug_application_time = None
inhibited_parameters=None
# Parameters for plotting
max_y_val =65
# adding burnin time to allow the system to reach steady state
burn_in_time = 0

In [ ]:
# Model Parameters and initial conditions as dictionaries

parameter_values = {
            'k_on': k_on,
            'k_off': k_off,
            'k_r': k_r,
            'k_t': kt,
            'k_p': k_p,
            'gamma_rn': gamma_r,
            'gamma_rc': gamma_r,
            'gamma_p': gamma_p,
        }

initial_conditions = {
    'G_off': 0,
    'G_on': 1,
    'R_n': 0,
    'R_c': 0,
    'P' : 0
}


In [ ]:
# Runnning the ODE model
time, concentrations_species =simulate_model(parameter_values, 
                                             initial_conditions, 
                                             total_simulation_time, 
                                             simulation_type='continuous', 
                                             burn_in_time=burn_in_time, 
                                             drug_application_time=drug_application_time, 
                                             inhibited_parameters=inhibited_parameters, )
plotting_deterministic(time,concentrations_species,species_colors,ylim_val=max_y_val)

In [ ]:
# Running SSA
time,trajectories_species  = simulate_model(parameter_values,
                                                  initial_conditions,
                                                  total_simulation_time,
                                                  simulation_type='discrete', 
                                                  burn_in_time=burn_in_time,
                                                  drug_application_time=drug_application_time,
                                                  inhibited_parameters=inhibited_parameters,
                                                  number_of_trajectories=number_of_trajectories)
plotting_combined(time, trajectories_species, species_colors,drug_application_time, time_point=80,ylim_val=max_y_val)

# Solving the model after drug application
____

In [ ]:
# Model Parameters
drug_application_time = 120
inhibition_constant = 0.1
inhibited_parameters={'k_t': effective_k_t*inhibition_constant}

In [ ]:
# Running the ODE model
time, concentrations_species = simulate_model(parameter_values,
                                                      initial_conditions,
                                                      total_simulation_time,
                                                      simulation_type='continuous', 
                                                      burn_in_time=burn_in_time,
                                                      drug_application_time=drug_application_time,
                                                      inhibited_parameters=inhibited_parameters)
plotting_deterministic(time,concentrations_species,species_colors,drug_application_time,ylim_val=max_y_val)

In [ ]:
# Running SSA
time, trajectories_species = simulate_model(parameter_values,
                                                  initial_conditions,
                                                  total_simulation_time,
                                                  simulation_type='discrete', 
                                                  burn_in_time=burn_in_time,
                                                  drug_application_time=drug_application_time,
                                                  inhibited_parameters=inhibited_parameters,
                                                  number_of_trajectories=number_of_trajectories)
plotting_stochastic(time, trajectories_species,species_colors,drug_application_time,ylim_val=max_y_val)

In [ ]:
xlim_vals = {'G_off': (-0.05, 1.05), 'G_on': (-0.05, 1.05), 'R_n': (0,60), 'R_c':(0,50), 'P': (0, 82)}  # Example x-axis limits for each species
plotting_stochastic_dist_all_species(time, trajectories_species, species_colors, time_point=75, xlim_vals=xlim_vals)
plotting_stochastic_dist_all_species(time, trajectories_species, species_colors, time_point=175, xlim_vals=xlim_vals)


# Parameter Estimation

___

In [ ]:
run_parameter_estimation = True
if run_parameter_estimation:
    pass
else:
    raise ValueError('Please set run_parameter_estimation to True to run the parameter estimation') 

In [ ]:
def adaptive_metropolis(log_target_pdf, start, chain_len, cov_matrix, rng=np.random.default_rng()):
    pdim = len(start)
    samples = np.zeros((chain_len, pdim))
    log_target_pdfs = np.zeros(chain_len)
    samples[0, :] = start
    log_target_pdfs[0] = log_target_pdf(samples[0, :])
    nacc = 0  # Number of accepted proposals
    cov_matrix_adjustment = 1.0e-6 * (2.4 ** 2.0) / pdim  # Adjustment factor for the covariance matrix
    update_freq = 50  # Frequency of covariance matrix updates
    pbar = tqdm(total=chain_len-1)
    for i in range(chain_len - 1):
        xpropose = rng.multivariate_normal(samples[i, :], cov_matrix)
        logpipropose = log_target_pdf(xpropose)
        logu = np.log(rng.uniform(0, 1))
        if logu <= logpipropose - log_target_pdfs[i]:
            samples[i + 1, :] = xpropose
            log_target_pdfs[i + 1] = logpipropose
            nacc += 1
        else:
            samples[i + 1, :] = samples[i, :]
            log_target_pdfs[i + 1] = log_target_pdfs[i]
        if (i + 1) % update_freq == 0 and i > 1:
            cov_matrix = np.cov(samples[:i + 1, :], rowvar=False) + cov_matrix_adjustment * np.eye(pdim)
        if (i + 1) % 100 == 0:  # Update progress bar every 100 iterations
            pbar.update(100)
            pbar.set_postfix(acceptance_rate=nacc / (i + 1))
    pbar.close()
    return samples, log_target_pdfs


In [ ]:
# True Model Parameters
k_on = 0.5
k_off = 0.1
k_r = 3
k_p = 0.9
gamma_r = 0.05 # assuming gamma_rn = gamma_rc
gamma_p = 0.65
transport_rate = 1
diffusion_rate = 10 # assuming diffusion_rate_r = diffusion_rate_p
total_simulation_time = 201
nucleus_diameter = 40
number_of_trajectories = 50
cytosol_diameter = 70 
model_type = '2D'

# Calculating effective transport rate
effective_k_t = calculate_effective_kt(D=nucleus_diameter, k_diff_r=diffusion_rate, transport_rate=transport_rate, model_type=model_type) 
print('Effective transport rate: ', effective_k_t)
kt = effective_k_t

In [ ]:
# updating parameter values
parameter_symbols = ['k_on', 'k_off', 'k_r', 'k_t', 'k_p', 'gamma_rn', 'gamma_rc', 'gamma_p','inhibition_constant']
true_parameter_values = [k_on, k_off, k_r, kt, k_p, gamma_r, gamma_r, gamma_p, inhibition_constant]
time_points = np.array([5,10,20,40,60,80,100,130,150,200]).astype(int)
chain_length = 1000
burnin_time =200
number_parameters= len(true_parameter_values)


In [ ]:
# Loading observable data
folder_simulated_data = current_dir.joinpath('simulated_data')
simulated_time_points = np.load(folder_simulated_data.joinpath('time_points_snapshots.npy'))
simulated_data_protein = np.load(folder_simulated_data.joinpath('snapshots_total_Protein_.npy'))
simulated_data_Rn = np.load(folder_simulated_data.joinpath('snapshots_RNA_nucleus.npy'))
simulated_data_Rc = np.load(folder_simulated_data.joinpath('snapshots_RNA_cytosol.npy'))

# calculating mean values and saving in List with shape (n_species, n_time_points). In the order Protein, RNA_nucleus, RNA_cytosol
observations_data = [ np.mean(simulated_data_protein, axis=0),
                        np.mean(simulated_data_Rn, axis=0),
                        np.mean(simulated_data_Rc, axis=0)]


In [ ]:
def Loglikelihood(  parameters, observations , drug_application_time=0, total_simulation_time=201, t_measurements=None):
    parameter_values = {
            'k_on': parameters[0],
            'k_off': parameters[1],
            'k_r': parameters[2],
            'k_t': parameters[3],
            'k_p': parameters[4],
            'gamma_rn': parameters[5],
            'gamma_rc': parameters[6],
            'gamma_p': parameters[7],
            'inhibition_constant': parameters[8]    
        }
    initial_conditions = {
        'G_off': 0,
        'G_on': 1,
        'R_n': 0,
        'R_c': 0,
        'P' : 0
    }
    inhibited_parameters={'k_t': parameter_values['k_t']*parameter_values['inhibition_constant']}
    time, concentrations_species =simulate_model(parameter_values, 
                                             initial_conditions, 
                                             total_simulation_time, 
                                             simulation_type='continuous', 
                                             burn_in_time=0, 
                                             drug_application_time=drug_application_time, 
                                             inhibited_parameters=inhibited_parameters, )
    if t_measurements is not None:
        y_R_n = concentrations_species['R_n'][t_measurements]
        y_R_c = concentrations_species['R_c'][t_measurements]
        y_P = concentrations_species['P'][t_measurements]
        SIGMA=1
        loglikelihood = 0.0
        for i in range(observations[0].shape[0]):
            loglikelihood +=    ( (np.sum(observations[0][i]) - y_P[i])**2/(2*SIGMA**2.0)  ) + ( (np.sum(observations[1][i]) - y_R_n[i])**2/(2*SIGMA**2.0) ) + ( (np.sum(observations[2][i]) - y_R_c[i])**2/(2*SIGMA**2.0) )
    else:
        loglikelihood = None
    return -loglikelihood, time, concentrations_species

In [ ]:
# initial values for the chain
BG_MU = np.array([-1.0, -1.0, -1.0, -1.0, -1.0,-1.0,-1.0,-1.0,-1.0])
BG_SIGMA = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
def bg_prior_sample(n_sample: int, rng=np.random.default_rng()) -> np.ndarray:
    in_range = False
    while in_range == False:
      prior_sample = rng.normal(loc=BG_MU, scale=BG_SIGMA, size=(n_sample, BG_MU.shape[0]))
      if np.any(10.0**(prior_sample) < 0.001) or np.any(10.0**(prior_sample) > 10):
          in_range= False
      else:
          in_range = True
    return prior_sample
def bg_logprior(parameter: np.ndarray):
    return -2.0*np.sum((parameter - BG_MU)**2.0 ) /(BG_SIGMA**2.0)
def bg_log_target(theta):
  transformed_theta = 10.0**(theta)
  if np.any(transformed_theta < 0.001) or np.any(transformed_theta > 100):
    return -1e200
  else:
    return bg_logprior(theta) + Loglikelihood(transformed_theta, observations=observations_data,drug_application_time=drug_application_time,total_simulation_time=201, t_measurements=time_points)[0]

In [ ]:
chain_start = bg_prior_sample(n_sample=1)[0]
print('inital guess:', 10.0**(chain_start))

In [28]:
run_ms = True
if run_ms:
    bg_chain, bg_logpos = adaptive_metropolis(bg_log_target,
                                            start=chain_start,
                                            chain_len=chain_length,
                                            cov_matrix=np.eye(BG_MU.shape[0]))
    np.save(folder_outputs.joinpath('bg_chain.npy'), bg_chain)
    np.save(folder_outputs.joinpath('bg_logpos.npy'), bg_logpos)
else:
    bg_chain = np.load(folder_outputs.joinpath('bg_chain.npy'))
    bg_logpos = np.load(folder_outputs.joinpath('bg_logpos.npy'))

In [ ]:
fig, axs = plt.subplots(2, 8, figsize=(20, 4))  # Adjusted for a horizontal layout
fig.set_tight_layout(True)
# Setting titles for the top row for "MCMC Chain" and the bottom row for "Autocorrelation"
for i in range(number_parameters):
    if i == 3:
        axs[0, i].set_title("MH Chain", fontsize=20)
        axs[1, i].set_title("Autocorrelation", fontsize=20)
    # MCMC chain plot
    axs[0, i].plot(bg_chain[burnin_time:, i], color='lightslategray', lw=2)
    axs[0, i].axhline(true_parameter_values[i], color='orangered', lw=3)
    axs[0, i].set_ylabel(parameter_symbols[i])
    axs[1, i].set_ylabel(parameter_symbols[i])
    axs[0, i].set_xlabel('MH-steps')
    axs[1, i].set_xlabel('MH-steps')
    # Autocorrelation plot
    # Note: Adjust the autocorrelation calculation based on your version of emcee
    autocorr = emcee.autocorr.function_1d(bg_chain[burnin_time:, i]) if hasattr(emcee.autocorr, 'function_1d') else np.correlate(bg_chain[burnin_time:, i], bg_chain[burnin_time:, i], mode='full')[len(bg_chain[burnin_time:, i])-1:] / len(bg_chain[burnin_time:, i])
    axs[1, i].plot(autocorr, color='lightslategray', lw=3)
fig.savefig(folder_outputs+"mcmc_trajectories_horizontal.png", dpi=400, bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15, 6))  # Adjusted to a 2x4 layout
fig.set_tight_layout(True)
# Loop through the parameter indices and plot
for i in range(8):
    row = i // 4  # Calculate row index
    col = i % 4  # Calculate column index
    # MCMC chain plot
    axs[row, col].plot(bg_chain[burnin_time:, i], color='lightslategray', lw=2)
    axs[row, col].axhline(true_parameter_values[i], color='orangered', lw=3)
    axs[row, col].set_ylabel(parameter_symbols[i])
    axs[row, col].set_xlabel('MH-steps')
# adding title for all plots
fig.suptitle("Metropolis-Hastings Chains", fontsize=20)
# Adjust the layout
fig.tight_layout()
fig.savefig(folder_outputs+"mcmc_trajectories_parameters.png", dpi=400, bbox_inches="tight")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15, 6))  # Adjusted to a 2x4 layout
fig.set_tight_layout(True)
# Loop through the parameter indices and create histograms
for i in range(8):
    row = i // 4  # Calculate row index
    col = i % 4  # Calculate column index
    data = bg_chain[burnin_time:, i]  # Extracting the burn-in adjusted chain for the parameter
    mean = np.mean(data)  # Calculate the mean of the parameter's chain
    ci_lower, ci_upper = np.percentile(data, [5, 95])  # Calculate the 90% CI
    # Histogram plot
    axs[row, col].hist(data, bins=30, color='lightslategray', alpha=0.7)
    axs[row, col].axvline(mean, color='orangered', lw=4, label='Estimate')
    axs[row, col].axvline(ci_lower, color='blue', linestyle='--', lw=2, label='90% CI')
    axs[row, col].axvline(ci_upper, color='blue', linestyle='--', lw=2)
    axs[row, col].axvline(true_parameter_values[i], color='green', lw=4, label='True', linestyle='--')  # Marking the true parameter value
    axs[row, col].set_ylabel('Frequency')
    axs[row, col].set_xlabel(parameter_symbols[i])
    axs[row, col].legend()
# Adding a title for all plots
fig.suptitle("Parameter Uncertainty", fontsize=20)
# Adjust the layout
fig.tight_layout()
# Save the figure
fig.savefig(folder_outputs+"mcmc_parameter_distributions_with_true.png", dpi=400, bbox_inches="tight")
plt.show()

In [ ]:
sns.set(style="white")  # Set Seaborn style
chain_trunc = bg_chain[burnin_time:, :]  # Remove burn-in samples
fig, axs = plt.subplots(number_parameters, number_parameters, figsize=(15, 15))
fig.tight_layout(pad=2.0)
for i in range(8):
    for j in range(i):
        axs[i, j].hexbin(chain_trunc[:, j], chain_trunc[:, i], gridsize=15, cmap='plasma', mincnt=1)
        axs[i, j].tick_params(axis='both', which='major', labelsize=8)
        if j == 0:
            axs[i, j].set_ylabel(parameter_symbols[i], fontsize=10)
        if i == 7:  # This ensures the x labels appear at the bottom row
            axs[i, j].set_xlabel(parameter_symbols[j], fontsize=10)
    for j in range(i + 1, len(true_parameter_values)):
        axs[i, j].axis('off')
    # Plot histogram
    n, bins, patches = axs[i, i].hist(chain_trunc[:, i], color='lightslategray', bins=10)
    axs[i, i].axvline(true_parameter_values[i], color="orangered", lw=4)
    axs[i, i].tick_params(axis='both', which='major', labelsize=8)
    # Use scientific notation for y-axis
    axs[i, i].yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
    axs[i, i].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.show()
fig.savefig(folder_outputs+"mcmc_distributions.png", dpi=400, bbox_inches="tight")

In [ ]:
# plot the ODE model with the best fit parameters
best_fit = np.mean(bg_chain[burnin_time:], axis=0)
print('best fit:', 10.0**(best_fit))
_, time, concentrations_species = Loglikelihood(best_fit, observations=observations_data,drug_application_time=drug_application_time,total_simulation_time=201, t_measurements=time_points)

# solving the model with the true parameters
_, time, concentrations_species_true = Loglikelihood(true_parameter_values, observations=observations_data,drug_application_time=drug_application_time,total_simulation_time=201, t_measurements=time_points)


In [ ]:
# Plotting the ODE model and the experimental data only for variables P and R_n
plt.figure(figsize=(8, 5))

# plotting for observations_data
plt.plot(time_points, observations_data[0], 'o', color=species_colors['P'], label=' Data (P)', markersize=8)
plt.plot(time_points, observations_data[1], 'o', color=species_colors['R_n'], label=' Data (R_n)', markersize=8)
plt.plot(time_points, observations_data[2], 'o', color=species_colors['R_c'], label=' Data (R_c)', markersize=8)

# plotting model fit
plt.plot(time, concentrations_species['P'],   color=species_colors['P'], label='Model Fit (P)', lw=4)
plt.plot(time, concentrations_species['R_n'], color=species_colors['R_n'], label='Model Fit (R_n)', lw=4)
plt.plot(time, concentrations_species['R_c'], color=species_colors['R_c'], label='Model Fit (R_c)', lw=4)

# plotting for concentrations_species_true
plt.plot(time, concentrations_species_true['P'], color=species_colors['P'], label='True Model (P)', lw=2, linestyle='--')
plt.plot(time, concentrations_species_true['R_n'], color=species_colors['R_n'], label='True Model (R_n)', lw=2, linestyle='--')
plt.plot(time, concentrations_species_true['R_c'], color=species_colors['R_c'], label='True Model (R_c)', lw=2, linestyle='--')

# Adding labels and legend
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.title('ODE Model vs Data')
plt.legend(loc='upper right', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.ylim(0, 55)
# save the figure

plt.savefig(folder_outputs+'model_fit.png', dpi=400, bbox_inches="tight")
plt.show()